# Importing requrie modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set seed

In [ ]:
seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)
plt.style.use('ggplot')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load data

In [ ]:
eth = pd.read_csv('./Data/ethusd_5-min_data.csv')
eth['Datetime'] = pd.to_datetime(eth['timestamp'], unit='s')
eth.set_index('Datetime', inplace=True)

# Plotting data

In [ ]:
fig = plt.figure(figsize=(16, 9))
plt.plot(eth.close)
plt.xlabel('Date')
plt.ylabel('Price')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
plt.title('ETH-USD')
plt.show()

# Scale data

In [ ]:
eth_scaler = MinMaxScaler()
eth_scaled = eth_scaler.fit_transform(pd.DataFrame(eth[['close']]))
eth_len = len(eth_scaled)
eth_train = eth_scaled[:int(eth_len * 0.8)]
eth_val = eth_scaled[int(eth_len * 0.8):int(eth_len * 0.9)]
eth_test = eth_scaled[int(eth_len * 0.9):]

# Create sliding windows

In [ ]:
def create_sliding_windows(data, lag, step=1):
    len_data = len(data)
    x = []
    y = []
    for i in range(lag * 2, len_data - lag, step):
        x.append(data[i - lag * 2:i, 0])
        y.append(data[i:i + lag, 0])
    return np.array(x), np.array(y)

train_x, train_y = create_sliding_windows(eth_train, 289)
val_x, val_y = create_sliding_windows(eth_val, 289)
test_x, test_y = create_sliding_windows(eth_test, 289, 289)

# Convert to PyTorch tensors

In [ ]:
train_x = torch.FloatTensor(train_x).unsqueeze(-1)
train_y = torch.FloatTensor(train_y)
val_x = torch.FloatTensor(val_x).unsqueeze(-1)
val_y = torch.FloatTensor(val_y)
test_x = torch.FloatTensor(test_x).unsqueeze(-1)
test_y = torch.FloatTensor(test_y)


# Create DataLoaders

In [ ]:
batch_size = 126 # 8192
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(val_x, val_y)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define GRU model

In [ ]:
class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(input_size=1, hidden_size=289, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.gru2 = nn.GRU(input_size=289, hidden_size=145, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.gru3 = nn.GRU(input_size=145, hidden_size=145, num_layers=1, batch_first=True)
        self.dropout3 = nn.Dropout(0.2)
        self.linear = nn.Linear(145, 289)
        
    def forward(self, x):
        x, _ = self.gru1(x)
        x = self.dropout1(x)
        x, _ = self.gru2(x)
        x = self.dropout2(x)
        x, _ = self.gru3(x)
        x = self.dropout3(x[:, -1, :])  # Take last output
        return self.linear(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRUModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

# Training loop

In [ ]:
def train_model(model, train_loader, val_loader, epochs, patience):
    best_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                output = model(x_val)
                val_loss += criterion(output, y_val).item()
        
        avg_val_loss = val_loss / len(val_loader)
        scheduler.step(avg_val_loss)
        
        # Early stopping
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            counter = 0
            torch.save(model.state_dict(), './model/best_model.pth')
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
        
        print(f'Epoch {epoch+1}: Train Loss: {train_loss/len(train_loader):.6f}, Val Loss: {avg_val_loss:.6f}')



In [ ]:
train_model(model, train_loader, val_loader, epochs=180, patience=10)